In [2]:
from vmc_torch.experiment.vmap.vmap_models import PEPS_Model
import quimb.tensor as qtn
import pickle
import torch
import numpy as np
import torch.nn as nn

Lx = 6
Ly = 6
D = 4
chi = 4*D
seed = 42
pwd = '/home/sijingdu/TNVMC/VMC_code/vmc_torch/vmc_torch/experiment/vmap/data'
skeleton = pickle.load(open(pwd+f'/{Lx}x{Ly}/heis/D={D}/peps_skeleton.pkl', 'rb'))
params = pickle.load(open(pwd+f'/{Lx}x{Ly}/heis/D={D}/peps_su_params.pkl', 'rb'))
peps = qtn.unpack(params, skeleton)
# peps = qtn.PEPS.rand(Lx, Ly, D, seed=seed)

model = PEPS_Model(tn=peps, max_bond=chi)

B = 64
random_binary_config = torch.randint(0, 2, (B, Lx*Ly)).int()
with torch.no_grad():
    print(model(random_binary_config))

tensor([-4.1779e-08,  9.7329e-12,  1.0413e-07, -7.5432e-09,  5.3392e-12,
        -2.9437e-06, -5.0227e-06,  5.1558e-08, -7.5467e-07,  7.4506e-06,
         8.1744e-09,  2.0786e-06,  6.1282e-06, -2.6822e-07,  8.2713e-13,
        -9.7471e-09, -4.5175e-09,  4.0028e-08, -1.2476e-07, -3.8701e-07,
        -1.0025e-06, -2.3055e-11, -4.9738e-09,  8.0886e-08, -8.8793e-10,
         1.7545e-06, -1.1090e-07,  3.9053e-10, -1.3652e-08,  1.0510e-07,
        -1.9913e-06, -6.0971e-06, -3.2986e-06, -1.0869e-07,  8.8661e-05,
         1.3513e-07,  1.2099e-06, -1.6036e-06,  1.1892e-06,  3.8461e-07,
         1.3023e-08, -2.1887e-06, -2.9337e-06, -2.3970e-07, -1.2801e-08,
        -3.9696e-06,  2.1027e-07,  1.4887e-06,  2.2544e-06, -4.5873e-07,
        -1.2213e-06, -2.1170e-05,  4.0730e-11,  1.9668e-07,  5.0741e-07,
         4.0429e-09, -4.2158e-07, -4.5128e-10,  6.3252e-08,  6.5152e-07,
         2.5725e-10, -3.9927e-07,  1.5364e-06,  1.0661e-07],
       dtype=torch.float64)


In [3]:
from vmc_torch.hamiltonian_torch import spin_Heisenberg_square_lattice_torch

H = spin_Heisenberg_square_lattice_torch(Lx, Ly, J=1.0, total_sz=0)
# all_states = torch.tensor(H.hilbert.all_states())
# psi_vec = model(all_states).detach().cpu().numpy()
# H_dense = H.to_dense()
# E_expect = np.vdot(psi_vec, H_dense @ psi_vec) / np.vdot(psi_vec, psi_vec)
# print("Energy per site:", E_expect)
# np.linalg.eigvals(H.to_dense())[0]/(Lx*Ly)  # ground state energy per site check

In [6]:
import quimb as qu
class PEPS_Model_reuse(nn.Module):
    def __init__(self, tn, max_bond, dtype=torch.float64):
        import quimb as qu
        import quimb.tensor as qtn
        super().__init__()
        
        params, skeleton = qtn.pack(tn)
        self.dtype = dtype
        self.skeleton = skeleton
        self.Lx = tn.Lx
        self.Ly = tn.Ly
        self.bMPS_x_skeletons = {}
        self.bMPS_y_skeletons = {}
        self.bMPS_params_x_in_dims = None
        self.bMPS_params_y_in_dims = None
        self.chi = max_bond

        # for torch, further flatten pytree into a single list
        params_flat, params_pytree = qu.utils.tree_flatten(
            params, get_ref=True
        )
        self.params_pytree = params_pytree

        # register the flat list parameters
        self.params = torch.nn.ParameterList([
            torch.as_tensor(x, dtype=self.dtype) for x in params_flat
        ])
    
    def cache_bMPS_skeleton(self, x):
        params = qu.utils.tree_unflatten(self.params, self.params_pytree)
        tn = qtn.unpack(params, self.skeleton)
        amp = tn.isel({tn.site_ind(site): x[i] for i, site in enumerate(tn.sites)})
        env_x = amp.compute_x_environments(max_bond=self.chi, cutoff=0.0)
        bMPS_params_dict = {}
        for key, tn in env_x.items():
            bMPS_params, skeleton = qtn.pack(tn)
            env_x[key] = skeleton
            bMPS_params_dict[key] = bMPS_params

        self.bMPS_x_skeletons = env_x
        bMPS_params_x_in_dims = qu.utils.tree_map(lambda _: 0, bMPS_params_dict)
        self.bMPS_params_x_in_dims = bMPS_params_x_in_dims

        env_y = amp.compute_y_environments(max_bond=self.chi, cutoff=0.0)
        bMPS_params_dict = {}
        for key, tn in env_y.items():
            bMPS_params, skeleton = qtn.pack(tn)
            env_y[key] = skeleton
            bMPS_params_dict[key] = bMPS_params
        self.bMPS_y_skeletons = env_y
        bMPS_params_y_in_dims = qu.utils.tree_map(lambda _: 0, bMPS_params_dict)
        self.bMPS_params_y_in_dims = bMPS_params_y_in_dims

    
    def cache_bMPS_params_vmap(self, x):
        # return a pytree (dict) of bMPS params for x and y environments
        params = qu.utils.tree_unflatten(self.params, self.params_pytree)
        def cache_bMPS_params_single(x_single, params):
            tn = qtn.unpack(params, self.skeleton)
            amp = tn.isel({tn.site_ind(site): x_single[i] for i, site in enumerate(tn.sites)})
            env_x = amp.compute_x_environments(max_bond=self.chi, cutoff=0.0)
            bMPS_params_x_dict = {}
            for key, btn in env_x.items():
                bMPS_params = btn.get_params()
                bMPS_params_x_dict[key] = bMPS_params
            bMPS_params_y_dict = {}
            env_y = amp.compute_y_environments(max_bond=self.chi, cutoff=0.0)
            for key, btn in env_y.items():
                bMPS_params = btn.get_params()
                bMPS_params_y_dict[key] = bMPS_params
            return bMPS_params_x_dict, bMPS_params_y_dict
        return torch.vmap(
            cache_bMPS_params_single,
            in_dims=(0, None),
        )(x, params)
    
    def cache_bMPS_params_any_direction_vmap(self, x, direction='x'):
        # return a pytree (dict) of bMPS params for x or y environments
        params = qu.utils.tree_unflatten(self.params, self.params_pytree)
        def cache_bMPS_params_x_single(x_single, params):
            tn = qtn.unpack(params, self.skeleton)
            amp = tn.isel({tn.site_ind(site): x_single[i] for i, site in enumerate(tn.sites)})
            env_x = amp.compute_x_environments(max_bond=self.chi, cutoff=0.0)
            amp_val = (env_x[('xmin', self.Lx//2)]|env_x[('xmax', self.Lx//2-1)]).contract()
            bMPS_params_x_dict = {}
            for key, btn in env_x.items():
                bMPS_params = btn.get_params()
                bMPS_params_x_dict[key] = bMPS_params
            return bMPS_params_x_dict, amp_val
        def cache_bMPS_params_y_single(x_single, params):
            tn = qtn.unpack(params, self.skeleton)
            amp = tn.isel({tn.site_ind(site): x_single[i] for i, site in enumerate(tn.sites)})
            env_y = amp.compute_y_environments(max_bond=self.chi, cutoff=0.0)
            amp_val = (env_y[('ymin', self.Ly//2)]|env_y[('ymax', self.Ly//2-1)]).contract()
            bMPS_params_y_dict = {}
            for key, btn in env_y.items():
                bMPS_params = btn.get_params()
                bMPS_params_y_dict[key] = bMPS_params
            return bMPS_params_y_dict, amp_val
        if direction == 'x':
            return torch.vmap(
                cache_bMPS_params_x_single,
                in_dims=(0, None),
            )(x, params)
        else:
            return torch.vmap(
                cache_bMPS_params_y_single,
                in_dims=(0, None),
            )(x, params)
    
    def update_bMPS_params_to_row_vmap(self, x, row_id, bMPS_params_x_batched, from_which='xmin'):
        # update the bMPS params to a specific row_id for all samples in the batch
        bMPS_key = (from_which, row_id)
        params = qu.utils.tree_unflatten(self.params, self.params_pytree)
        def update_bMPS_params_x_single(x_single, params, row_id, bMPS_params_x, from_which):
            tn = qtn.unpack(params, self.skeleton)
            amp = tn.isel({tn.site_ind(site): x_single[i] for i, site in enumerate(tn.sites)})
            bMPS_to_row = qtn.unpack(bMPS_params_x[bMPS_key], self.bMPS_x_skeletons[bMPS_key])
            row_tn = amp.select([tn.row_tag(row_id)], which='any')
            # MPO-MPS two row TN
            updated_bMPS = (bMPS_to_row|row_tn)
            # contract to get the updated bMPS, row_id+1 for xmin, row_id-1 for xmax
            if from_which == 'xmin':
                if row_id == 0:
                    updated_bMPS = row_tn
                else:
                    updated_bMPS.contract_boundary_from_xmin_(max_bond=self.chi, cutoff=0.0, xrange=[row_id-1, row_id])
                updated_bMPS_params = updated_bMPS.get_params()
                pytree_params, _ = qu.utils.tree_flatten(updated_bMPS_params, get_ref=True)
                _, pytree = qu.utils.tree_flatten(bMPS_params_x[(from_which, row_id+1)], get_ref=True)
                updated_bMPS_params = qu.utils.tree_unflatten(pytree_params, pytree)
                bMPS_params_x[(from_which, row_id+1)] = updated_bMPS_params # inplace update
            else:
                if row_id == amp.Ly-1:
                    updated_bMPS = row_tn
                else:
                    updated_bMPS.contract_boundary_from_xmax_(max_bond=self.chi, cutoff=0.0, xrange=[row_id, row_id+1])
                updated_bMPS_params = updated_bMPS.get_params()
                pytree_params, _ = qu.utils.tree_flatten(updated_bMPS_params, get_ref=True)
                _, pytree = qu.utils.tree_flatten(bMPS_params_x[(from_which, row_id-1)], get_ref=True)
                updated_bMPS_params = qu.utils.tree_unflatten(pytree_params, pytree)
                bMPS_params_x[(from_which, row_id-1)] = updated_bMPS_params # inplace update
            return bMPS_params_x
        return torch.vmap(
            update_bMPS_params_x_single,
            in_dims=(0, None, None, self.bMPS_params_x_in_dims, None),
        )(x, params, row_id, bMPS_params_x_batched, from_which)
    
    def update_bMPS_params_to_col_vmap(self, x, col_id, bMPS_params_y_batched, from_which='ymin'):
        # update the bMPS params to a specific col_id for all samples in the batch
        bMPS_key = (from_which, col_id)
        params = qu.utils.tree_unflatten(self.params, self.params_pytree)
        def update_bMPS_params_y_single(x_single, params, col_id, bMPS_params_y, from_which):
            tn = qtn.unpack(params, self.skeleton)
            amp = tn.isel({tn.site_ind(site): x_single[i] for i, site in enumerate(tn.sites)})
            bMPS_to_col = qtn.unpack(bMPS_params_y[bMPS_key], self.bMPS_y_skeletons[bMPS_key])
            col_tn = amp.select([tn.col_tag(col_id)], which='any')
            # MPO-MPS two col TN
            updated_bMPS = (bMPS_to_col|col_tn)
            # contract to get the updated bMPS, col_id+1 for ymin, col_id-1 for ymax
            if from_which == 'ymin':
                if col_id == 0:
                    updated_bMPS = col_tn
                else:
                    updated_bMPS.contract_boundary_from_ymin_(max_bond=self.chi, cutoff=0.0, yrange=[col_id-1, col_id])
                updated_bMPS_params = updated_bMPS.get_params()
                pytree_params, _ = qu.utils.tree_flatten(updated_bMPS_params, get_ref=True)
                _, pytree = qu.utils.tree_flatten(bMPS_params_y[(from_which, col_id+1)], get_ref=True)
                updated_bMPS_params = qu.utils.tree_unflatten(pytree_params, pytree)
                bMPS_params_y[(from_which, col_id+1)] = updated_bMPS_params # inplace update
            else:
                if col_id == amp.Lx-1:
                    updated_bMPS = col_tn
                else:
                    updated_bMPS.contract_boundary_from_ymax_(max_bond=self.chi, cutoff=0.0, yrange=[col_id, col_id+1])
                updated_bMPS_params = updated_bMPS.get_params()
                pytree_params, _ = qu.utils.tree_flatten(updated_bMPS_params, get_ref=True)
                _, pytree = qu.utils.tree_flatten(bMPS_params_y[(from_which, col_id-1)], get_ref=True)
                updated_bMPS_params = qu.utils.tree_unflatten(pytree_params, pytree)
                bMPS_params_y[(from_which, col_id-1)] = updated_bMPS_params # inplace update
            return bMPS_params_y
        return torch.vmap(
            update_bMPS_params_y_single,
            in_dims=(0, None, None, self.bMPS_params_y_in_dims, None),
        )(x, params, col_id, bMPS_params_y_batched, from_which)
            
        
    def amp_tn(self, x):
        params = qu.utils.tree_unflatten(self.params, self.params_pytree)
        tn = qtn.unpack(params, self.skeleton)
        # might need to specify the right site ordering here
        amp = tn.isel({tn.site_ind(site): x[i] for i, site in enumerate(tn.sites)})
        return amp
    
    def amplitude(
        self,
        x,
        params,
        bMPS_keys=None,
        bMPS_params_xmin=None,
        bMPS_params_xmax=None,
        bMPS_params_ymin=None,
        bMPS_params_ymax=None,
        selected_rows=None,
        selected_cols=None,
    ):
        tn = qtn.unpack(params, self.skeleton)
        # might need to specify the right site ordering here
        amp = tn.isel({tn.site_ind(site): x[i] for i, site in enumerate(tn.sites)})

        # replace the x-environment with the cached one
        if bMPS_params_xmin is not None and bMPS_params_xmax is not None and bMPS_keys is not None:
            bMPS_min = qtn.unpack(bMPS_params_xmin, self.bMPS_x_skeletons[bMPS_keys[0]])
            bMPS_max = qtn.unpack(bMPS_params_xmax, self.bMPS_x_skeletons[bMPS_keys[1]])
            rows = amp.select([tn.row_tag(row) for row in selected_rows], which='any')
            amp_reuse = (bMPS_min|rows|bMPS_max)
            amp_reuse.view_as_(
                qtn.PEPS,
                site_tag_id = tn._site_tag_id,
                x_tag_id = tn._x_tag_id,
                y_tag_id = tn._y_tag_id,
                Lx = tn._Lx,
                Ly = tn._Ly,
                site_ind_id = tn._site_ind_id,
            )
            if self.chi > 0:
                amp_reuse.contract_boundary_from_xmin_(max_bond=self.chi, cutoff=0.0, xrange=[bMPS_keys[0][1], bMPS_keys[1][1]+1])
            return amp_reuse.contract()
        # replace the y-environment with the cached one
        if bMPS_params_ymin is not None and bMPS_params_ymax is not None and bMPS_keys is not None:
            bMPS_min = qtn.unpack(bMPS_params_ymin, self.bMPS_y_skeletons[bMPS_keys[0]])
            bMPS_max = qtn.unpack(bMPS_params_ymax, self.bMPS_y_skeletons[bMPS_keys[1]])
            cols = amp.select([tn.col_tag(col) for col in selected_cols], which='any')
            amp_reuse = (bMPS_min|cols|bMPS_max)
            amp_reuse.view_as_(
                qtn.PEPS,
                site_tag_id = tn._site_tag_id,
                x_tag_id = tn._x_tag_id,
                y_tag_id = tn._y_tag_id,
                Lx = tn._Lx,
                Ly = tn._Ly,
                site_ind_id = tn._site_ind_id,
            )
            if self.chi > 0:
                amp_reuse.contract_boundary_from_ymin_(max_bond=self.chi, cutoff=0.0, yrange=[bMPS_keys[0][1], bMPS_keys[1][1]+1])
            return amp_reuse.contract()

        if self.chi > 0:
            amp.contract_boundary_from_ymin_(max_bond=self.chi, cutoff=0.0, yrange=[0, amp.Ly//2-1])
            amp.contract_boundary_from_ymax_(max_bond=self.chi, cutoff=0.0, yrange=[amp.Ly//2, amp.Ly-1])
        return amp.contract()
    
    def vamp(
        self,
        x,
        params,
        bMPS_keys=None,
        bMPS_params_xmin=None,
        bMPS_params_xmax=None,
        bMPS_params_ymin=None,
        bMPS_params_ymax=None,
        selected_rows=None,
        selected_cols=None,
    ):
        params = qu.utils.tree_unflatten(params, self.params_pytree)
        if bMPS_params_xmin is not None and bMPS_params_xmax is not None:
            return torch.vmap(
                self.amplitude,
                in_dims=(
                    0,
                    None,
                    None,
                    self.bMPS_params_x_in_dims[bMPS_keys[0]],
                    self.bMPS_params_x_in_dims[bMPS_keys[1]],
                    None,
                    None,
                    None,
                    None,
                ),
            )(x, params, bMPS_keys, bMPS_params_xmin, bMPS_params_xmax, bMPS_params_ymin, bMPS_params_ymax, selected_rows, selected_cols)
        
        if bMPS_params_ymin is not None and bMPS_params_ymax is not None:
            return torch.vmap(
                self.amplitude,
                in_dims=(
                    0,
                    None,
                    None,
                    None,
                    None,
                    self.bMPS_params_y_in_dims[bMPS_keys[0]],
                    self.bMPS_params_y_in_dims[bMPS_keys[1]],
                    None,
                    None,
                ),
            )(x, params, bMPS_keys, bMPS_params_xmin, bMPS_params_xmax, bMPS_params_ymin, bMPS_params_ymax, selected_rows, selected_cols)

        def amplitude_det(x, params):
            tn = qtn.unpack(params, self.skeleton)
            # might need to specify the right site ordering here
            amp = tn.isel({tn.site_ind(site): x[i] for i, site in enumerate(tn.sites)})
            if self.chi > 0:
                amp.contract_boundary_from_ymin_(max_bond=self.chi, cutoff=0.0, yrange=[0, amp.Ly//2-1])
                amp.contract_boundary_from_ymax_(max_bond=self.chi, cutoff=0.0, yrange=[amp.Ly//2, amp.Ly-1])
            return amp.contract()
        
        return torch.vmap(
            amplitude_det,
            in_dims=(0, None),
        )(x, params)
    
    def forward(
        self,
        x,
        bMPS_params_x_batched=None,
        bMPS_params_y_batched=None,
        selected_rows=None,
        selected_cols=None,
    ):
        bMPS_params_xmin = None
        bMPS_params_xmax = None
        bMPS_params_ymin = None
        bMPS_params_ymax = None
        bMPS_keys = None

        if selected_rows is not None:
            bMPS_keys = [('xmin', min(selected_rows)), ('xmax', max(selected_rows))]
            bMPS_params_xmin = bMPS_params_x_batched[bMPS_keys[0]]
            bMPS_params_xmax = bMPS_params_x_batched[bMPS_keys[1]]
        if selected_cols is not None:
            bMPS_keys = [('ymin', min(selected_cols)), ('ymax', max(selected_cols))]
            bMPS_params_ymin = bMPS_params_y_batched[bMPS_keys[0]]
            bMPS_params_ymax = bMPS_params_y_batched[bMPS_keys[1]]
        
        return self.vamp(
            x,
            self.params,
            bMPS_keys=bMPS_keys,
            bMPS_params_xmin=bMPS_params_xmin,
            bMPS_params_xmax=bMPS_params_xmax,
            bMPS_params_ymin=bMPS_params_ymin,
            bMPS_params_ymax=bMPS_params_ymax,
            selected_rows=selected_rows,
            selected_cols=selected_cols,
        )

model_reuse = PEPS_Model_reuse(tn=peps, max_bond=chi)
model_reuse.cache_bMPS_skeleton(random_binary_config[0])
reuse_B = 100
test_fxs = random_binary_config[:reuse_B]
amp_tns = [model_reuse.amp_tn(test_fx) for test_fx in test_fxs]
with torch.no_grad():
    B_bMPS_params_x_dict, B_bMPS_params_y_dict = model_reuse.cache_bMPS_params_vmap(test_fxs)
    current_amps = model_reuse(test_fxs, bMPS_params_x_batched=B_bMPS_params_x_dict, bMPS_params_y_batched=B_bMPS_params_y_dict, selected_rows=(2,), selected_cols=None)
# current_amps
# [amp_tns[i].contract() for i in range(reuse_B)]
# print(test_fxs)
# print("Amplitudes with reuse:", current_amps)
model_reuse(test_fxs)

/home/sijingdu/TNVMC/VMC_code/quimb/quimb/utils.py:170: Warning: The row_tag function is deprecated in favor of x_tag
  warnings.warn(


tensor([-4.1779e-08,  9.7329e-12,  1.0413e-07, -7.5432e-09,  5.3392e-12,
        -2.9437e-06, -5.0227e-06,  5.1558e-08, -7.5467e-07,  7.4506e-06,
         8.1744e-09,  2.0786e-06,  6.1282e-06, -2.6822e-07,  8.2713e-13,
        -9.7471e-09, -4.5175e-09,  4.0028e-08, -1.2476e-07, -3.8701e-07,
        -1.0025e-06, -2.3055e-11, -4.9738e-09,  8.0886e-08, -8.8793e-10,
         1.7545e-06, -1.1090e-07,  3.9053e-10, -1.3652e-08,  1.0510e-07,
        -1.9913e-06, -6.0971e-06, -3.2986e-06, -1.0869e-07,  8.8661e-05,
         1.3513e-07,  1.2099e-06, -1.6036e-06,  1.1892e-06,  3.8461e-07,
         1.3023e-08, -2.1887e-06, -2.9337e-06, -2.3970e-07, -1.2801e-08,
        -3.9696e-06,  2.1027e-07,  1.4887e-06,  2.2544e-06, -4.5873e-07,
        -1.2213e-06, -2.1170e-05,  4.0730e-11,  1.9668e-07,  5.0741e-07,
         4.0429e-09, -4.2158e-07, -4.5128e-10,  6.3252e-08,  6.5152e-07,
         2.5725e-10, -3.9927e-07,  1.5364e-06,  1.0661e-07],
       dtype=torch.float64, grad_fn=<MulBackward0>)

In [528]:
changed_row_id = 1
# randomly permute the row 0 of all test_fxs
# torch.random.manual_seed(42)
# perm = torch.randperm(Ly)
# test_fxs[:, changed_row_id*Ly:(changed_row_id+1)*Ly] = test_fxs[:, changed_row_id*Ly:(changed_row_id+1)*Ly][:, perm]
# print(test_fxs)

with torch.no_grad():
    new_B_bMPS_params_x_dict = model_reuse.update_bMPS_params_to_row_vmap(test_fxs, row_id=1, bMPS_params_x_batched=B_bMPS_params_x_dict, from_which='xmin')

# get the pytree structure of new_B_bMPS_params_x_dict[('xmin', 2)]
with torch.no_grad():
    pytree = torch.utils._pytree.tree_flatten(new_B_bMPS_params_x_dict[('xmin', 2)])[1]
    pytree_old = torch.utils._pytree.tree_flatten(B_bMPS_params_x_dict[('xmin', 2)])[1]
    current_amps = model_reuse(test_fxs, bMPS_params_x_batched=new_B_bMPS_params_x_dict, bMPS_params_y_batched=B_bMPS_params_y_dict, selected_rows=(2,), selected_cols=None)
# current_amps
# print("Amplitudes with reuse after updating row 1:", current_amps)

In [ ]:
import mpi4py.MPI as MPI
import time
COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()

# cache skeleton for reuse
model_reuse.cache_bMPS_skeleton(test_fxs[0]) 
@torch.inference_mode()
def evaluate_energy_reuse(fxs, v_model, H, current_amps, verbose=False, benchmark_model=None):
    t0 = time.time()
    # Label each connected config with which sample it comes from to enable reuse
    B = fxs.shape[0]
    # cache bMPS params for reuse
    B_bMPS_params_x_dict, B_bMPS_params_y_dict = v_model.cache_bMPS_params_vmap(fxs)

    def detect_changed_row_col_pair(fx1, fx2):
        # currently only support nearest neighbor on square lattice
        Ly = v_model.skeleton._Ly
        changed_pos = torch.nonzero(fx1 - fx2)
        changed_pos_2d = []
        assert changed_pos.shape[0] <= 2, "Expect at most 2 on-site config changes"
        for pos in changed_pos:
            x, y = pos.item() // Ly, pos.item() % Ly
            changed_pos_2d.append( (x, y) )
        if len(changed_pos_2d) == 2:
            delta_row = abs(changed_pos_2d[0][0] - changed_pos_2d[1][0])
            delta_col = abs(changed_pos_2d[0][1] - changed_pos_2d[1][1])
            if delta_row <= delta_col:
                x1 = min(changed_pos_2d, key=lambda t: t[0])[0]
                row = True
                col = False
                return row, col, list(x for x in range(x1, x1+delta_row+1))
            else:
                y1 = min(changed_pos_2d, key=lambda t: t[1])[1]
                row = False
                col = True
                return row, col, list(y for y in range(y1, y1+delta_col+1))
        else:
            row = col = False
            return row, col, None
             
    # --------------------------------------------------------------------------
    # 2. Batched Calculation with Reuse
    # --------------------------------------------------------------------------
    # get connected configurations, coefficients and indices
    conn_eta_num = []
    conn_etas = []
    conn_eta_coeffs = []
    conn_eta_indices = []

    fx_ind = 0
    for fx in fxs:
        eta, coeffs = H.get_conn(fx)
        for i_eta in range(len(eta)):
            r, c, pos = detect_changed_row_col_pair(fx, eta[i_eta])
            conn_eta_indices.append( (fx_ind, i_eta, r, c, pos) )

        conn_eta_num.append(len(eta))
        conn_etas.append(torch.tensor(eta))
        conn_eta_coeffs.append(torch.tensor(coeffs))
        fx_ind += 1

    conn_etas = torch.cat(conn_etas, dim=0)
    conn_eta_coeffs = torch.cat(conn_eta_coeffs, dim=0)

    # group connected configs by changed row/col first
    # within row/col group, further group by position

    # select the indices where r==True and c==False
    # TODO: modify this part, form the pytree with batched leaves to input to reusbale PEPS amplitude calculation [x]
    tasks_map = {} # Key: (mode, indices_tuple), Value: lists of (global_idx, parent_idx)
    
    # 记录哪些是“对角项”（x' == x），这些不需要重算，直接复用当前振幅
    diagonal_mask = torch.zeros(conn_etas.shape[0], dtype=torch.bool, device=fxs.device)
    diagonal_parent_indices = []
    
    # 遍历所有连接构型，进行归类
    # conn_eta_indices[k] = (parent_fx_ind, i_eta, r_bool, c_bool, pos_list)
    for k, (parent_idx, _, is_row, is_col, indices) in enumerate(conn_eta_indices):
        if indices is None:
            # Config 没有变化 (Diagonal term, e.g. density-density interaction)
            diagonal_mask[k] = True
            diagonal_parent_indices.append(parent_idx)
            continue
            
        mode = 'row' if is_row else 'col'
        # 将 list 转为 tuple 以便作为 dict key
        indices_tuple = tuple(sorted(indices)) 
        group_key = (mode, indices_tuple)
        
        if group_key not in tasks_map:
            tasks_map[group_key] = {'global_idxs': [], 'parent_idxs': []}
        
        tasks_map[group_key]['global_idxs'].append(k)
        tasks_map[group_key]['parent_idxs'].append(parent_idx)

    # --------------------------------------------------------------------------
    # 2. Batched Calculation with Reuse
    # --------------------------------------------------------------------------
    # 预分配结果容器
    total_conns = conn_etas.shape[0]
    conn_amps = torch.zeros(total_conns, dtype=current_amps.dtype, device=fxs.device)
    
    # A. 处理对角项 (Direct Copy)
    if len(diagonal_parent_indices) > 0:
        # 找出所有对角项在 conn_amps 中的位置
        diag_locs = torch.nonzero(diagonal_mask).squeeze()
        # 找出对应的 parent index
        parents = torch.tensor(diagonal_parent_indices, device=fxs.device)
        # 直接赋值: <x|psi>
        conn_amps[diag_locs] = current_amps[parents]

    # B. 处理非对角项 (Grouped Vmap Contraction)
    # Fetch the corresponding Batched Environment Dictionary in the pytree
    def slice_env_dict(env_dict, idxs):
        """
        env_dict: {key: PyTree_of_Tensors}
        idxs: indices to slice (tensor or list)
        
        我们需要对 env_dict 中的每一个 value (PyTree) 进行操作，
        利用 qu.utils.tree_map 进入 PyTree 内部，对每个叶子 Tensor 进行切片。
        """
        return {
            k: qu.utils.tree_map(lambda x: x[idxs], v) 
            for k, v in env_dict.items()
        }

    for (mode, indices), data in tasks_map.items():
        # 获取索引
        global_idxs = data['global_idxs']  # 在 conn_etas 中的位置
        parent_idxs = data['parent_idxs']  # 来源于哪个 fxs[i]
        
        # 构造当前 Group 的 Batch
        # 1. 目标构型 x'
        # 注意：conn_etas 是一个大 tensor，直接切片即可
        target_configs = conn_etas[global_idxs] # (Batch_Group, N_sites)
        
        # 2. 对应的父辈索引 (用于取环境)
        subset_parents = torch.tensor(parent_idxs, device=fxs.device)
        
        # 3. 切片环境参数
        subset_env_x = slice_env_dict(B_bMPS_params_x_dict, subset_parents)
        subset_env_y = slice_env_dict(B_bMPS_params_y_dict, subset_parents)
        
        # 4. 调用模型 (Reuse Forward)
        # 这里利用了 v_model.forward 的接口
        if mode == 'row':
            amps_group = v_model(
                target_configs,
                bMPS_params_x_batched=subset_env_x,
                bMPS_params_y_batched=subset_env_y,
                selected_rows=indices,
                selected_cols=None
            )
        else: # col
            amps_group = v_model(
                target_configs,
                bMPS_params_x_batched=subset_env_x,
                bMPS_params_y_batched=subset_env_y,
                selected_rows=None,
                selected_cols=indices
            )
            
        # 5. 填回结果 Tensor
        locs = torch.tensor(global_idxs, device=fxs.device)
        conn_amps[locs] = amps_group

    # --------------------------------------------------------------------------
    # 3. Compute Local Energy
    # --------------------------------------------------------------------------
    # E_loc(x) = \sum_{x'} H_{x,x'} * (psi(x') / psi(x))
    
    # 此时 conn_amps 已经填满，且顺序与 conn_etas 一致
    # conn_eta_num[b] 告诉我们每个样本 b 有多少个连接构型
    
    local_energies = []
    offset = 0
    
    # 为了避免显式 Python 循环 (虽然 B=1024 时循环也不慢)，可以使用 segment_coo 或者简单的 loop
    # 简单 loop 实现：
    for b in range(B):
        n_conn = conn_eta_num[b]
        
        # 取出当前样本相关的所有连接构型的振幅
        # shape: (n_conn,)
        amps_slice = conn_amps[offset : offset + n_conn]
        coeffs_slice = conn_eta_coeffs[offset : offset + n_conn]
        
        # Ratio: psi(x') / psi(x)
        # current_amps[b] 是标量
        ratio = amps_slice / current_amps[b]
        
        # H_loc = \sum H_{xx'} * Ratio
        e_loc = torch.sum(coeffs_slice * ratio)
        
        local_energies.append(e_loc)
        offset += n_conn
        
    local_energies = torch.stack(local_energies)
    
    # Global Mean Energy
    energy_mean = torch.mean(local_energies)

    # if verbose and torch.distributed.is_initialized() and torch.distributed.get_rank() == 0:
    #     print(f"Reuse Stats: Processed {len(tasks_map)} groups + diagonal terms.")

    if benchmark_model is not None and verbose:
        t1 = time.time()
        print(f'Energy (Reuse)    : {energy_mean.item()}, time: {t1 - t0:.4f} s')
        # below is logics without reuse implemented yet
        ################################################################################
        current_amps_benchmark = benchmark_model(fxs)
        t0 = time.time()
        # calculate amplitudes for connected configs, in the future consider TN reuse to speed up calculation, TN reuse is controlled by a param that is not batched over (control flow?)
        conn_amps_benchmark = torch.cat([benchmark_model(conn_etas[i:i+B]) for i in range(0, conn_etas.shape[0], B)])

        # Local energy \sum_{s'} H_{s,s'} <s'|psi>/<s|psi>

        local_energies = []
        offset = 0
        for b in range(B):
            n_conn = conn_eta_num[b]
            amps_ratio = conn_amps_benchmark[offset:offset+n_conn] / current_amps_benchmark[b]
            energy_b = torch.sum(conn_eta_coeffs[offset:offset+n_conn] * amps_ratio)
            local_energies.append(energy_b)
            offset += n_conn
        local_energies = torch.stack(local_energies, dim=0)
        # Energy: (1/N) * \sum_s <s|H|psi>/<s|psi> = (1/N) * \sum_s \sum_{s'} H_{s,s'} <s'|psi>/<s|psi>
        energy = torch.mean(local_energies)
        t1 = time.time()
        print(f'Energy (Benchmark): {energy.item()}, time: {t1 - t0:.4f} s')
        ################################################################################

    return energy_mean, local_energies
    

energy_mean, local_energies = evaluate_energy_reuse(
    test_fxs, model_reuse, H, current_amps, verbose=True, benchmark_model=model
)
print(f'mean energy std: {local_energies.std().item()/np.sqrt(reuse_B)}')

/tmp/ipykernel_1386/881000176.py:19: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  changed_pos = torch.nonzero(fx1 - fx2)


Energy (Reuse)    : -19.618847917040775, time: 2.1739 s
Energy (Benchmark): -19.618848144724087, time: 6.0998 s
mean energy std: 0.2823158582856436


In [541]:
import mpi4py.MPI as MPI
import time
import random
from vmc_torch.experiment.vmap.vmap_utils import propose_exchange_or_hopping, sample_next
COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()

@torch.inference_mode()
def sample_next_reuse(fxs, v_model, graph, hopping_rate=0.25, verbose=False, seed=None, benchmark_model=None):
    B = fxs.shape[0]
    # cache bMPS params along x direction first for reuse
    B_bMPS_params_x_dict, current_amps = v_model.cache_bMPS_params_any_direction_vmap(fxs, direction='x')
    for row, edges in graph.row_edges.items():
        for edge in edges:
            i, j = edge
            proposed_fxs, new_flags = [], []
            fx_id = 0
            for fx in fxs:
                proposed_fx, new = propose_exchange_or_hopping(
                    i,
                    j,
                    fx,
                    hopping_rate=hopping_rate,
                    seed=seed + fx_id if seed is not None else None,
                )
                proposed_fxs.append(proposed_fx)
                new_flags.append(new)
                fx_id += 1
            proposed_fxs = torch.stack(proposed_fxs, dim=0)
            if not any(new_flags):
                if verbose:
                    print(f"No changes proposed in this edge. (x, {row}, edge: {edge})")
                continue
            # compute amplitudes for all proposed configs (because must align batchsize with reused bMPS batchsize)
            new_proposed_fxs = proposed_fxs
            # reuse of bMPS from xmin & xmax w.r.t. row to compute amplitudes
            new_proposed_amps = v_model(
                new_proposed_fxs,
                bMPS_params_x_batched=B_bMPS_params_x_dict,
                bMPS_params_y_batched=None,
                selected_rows=(row,),
                selected_cols=None,
            )
            proposed_amps = new_proposed_amps
            if benchmark_model is not None and verbose:
                benchmark_amps = benchmark_model(new_proposed_fxs)
                print(f"Benchmark vs Reuse amplitudes check (x, {row}):")
                print(torch.allclose(benchmark_amps, new_proposed_amps, atol=1e-5))
            # Metropolis-Hastings acceptance
            ratio = proposed_amps**2 / current_amps**2
            accept_probs = torch.minimum(ratio, torch.ones_like(ratio))
            for k in range(B):
                if random.random() < accept_probs[k].item():
                    fxs[k] = proposed_fxs[k]
                    current_amps[k] = proposed_amps[k]
        
        # update bMPS params to next row for reuse
        if row == v_model.Lx - 1:
            # reach the bottom row, no further bMPS to be updated
            break
        B_bMPS_params_x_dict = v_model.update_bMPS_params_to_row_vmap(
            fxs,
            row_id = row,
            bMPS_params_x_batched=B_bMPS_params_x_dict,
            from_which='xmin',
        )

    B_bMPS_params_y_dict, current_amps = v_model.cache_bMPS_params_any_direction_vmap(fxs, direction='y')
    for col, edges in graph.col_edges.items():
        for edge in edges:
            i, j = edge
            proposed_fxs, new_flags = [], []
            fx_id = 0
            for fx in fxs:
                proposed_fx, new = propose_exchange_or_hopping(
                    i,
                    j,
                    fx,
                    hopping_rate=hopping_rate,
                    seed=seed + fx_id if seed is not None else None,
                )
                proposed_fxs.append(proposed_fx)
                new_flags.append(new)
                fx_id += 1
            proposed_fxs = torch.stack(proposed_fxs, dim=0)
            if not any(new_flags):
                if verbose:
                    print(f"No changes proposed in this edge. (y, {col}, edge: {edge})")
                continue
            # compute amplitudes for all proposed configs (because must align batchsize with reused bMPS batchsize)
            new_proposed_fxs = proposed_fxs
            # reuse of bMPS from ymin & ymax w.r.t. col to compute amplitudes
            new_proposed_amps = v_model(
                new_proposed_fxs,
                bMPS_params_x_batched=None,
                bMPS_params_y_batched=B_bMPS_params_y_dict,
                selected_rows=None,
                selected_cols=(col,),
            )
            if benchmark_model is not None and verbose:
                benchmark_amps = benchmark_model(new_proposed_fxs)
                print(f"Benchmark vs Reuse amplitudes check (y, {col}):")
                print(torch.allclose(benchmark_amps, new_proposed_amps, atol=1e-5))
            proposed_amps = new_proposed_amps
            # Metropolis-Hastings acceptance
            ratio = proposed_amps**2 / current_amps**2
            accept_probs = torch.minimum(ratio, torch.ones_like(ratio))
            for k in range(B):
                if random.random() < accept_probs[k].item():
                    fxs[k] = proposed_fxs[k]
                    current_amps[k] = proposed_amps[k]
        
        # update bMPS params to next col for reuse
        if col == v_model.Ly - 1:
            # reach the rightmost col, no further bMPS to be updated
            break
        B_bMPS_params_y_dict = v_model.update_bMPS_params_to_col_vmap(
            fxs,
            col_id = col,
            bMPS_params_y_batched=B_bMPS_params_y_dict,
            from_which='ymin',
        )

    return fxs, current_amps

t0 = time.time()
test_fxs, current_amps = sample_next_reuse(
    test_fxs,
    model_reuse,
    H.graph,
    hopping_rate=0.0,
    verbose=False,
    seed=42,
    benchmark_model=model
)
t1 = time.time()
print(f"Sampling with reuse done. Time: {t1 - t0:.4f} s")

t0 = time.time()
test_fxs, current_amps = sample_next(
    test_fxs,
    model,
    H.graph,
    hopping_rate=0.0,
    verbose=False,
    seed=42,
)
t1 = time.time()
print(f"Sampling benchmark done. Time: {t1 - t0:.4f} s")

Sampling with reuse done. Time: 3.1913 s
Sampling benchmark done. Time: 6.3457 s


In [10]:
_EXTRA_PROPS = ('_site_tag_id', '_x_tag_id', '_y_tag_id', '_Lx', '_Ly', '_site_ind_id')
# get the extra properties of the tensor network
qtn_extra_props = {prop: getattr(peps, prop) for prop in _EXTRA_PROPS}
qtn_extra_props

{'_site_tag_id': 'I{},{}',
 '_x_tag_id': 'X{}',
 '_y_tag_id': 'Y{}',
 '_Lx': 6,
 '_Ly': 6,
 '_site_ind_id': 'k{},{}'}